In [1]:
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go

def bet_eval_metrics(d):

    metrics = pd.DataFrame(d
    .agg({"gpl": "sum", "stake": "sum"})
    ).transpose().assign(pot=lambda x: x['gpl'] / x['stake'])

    return(metrics[metrics['stake'] != 0])

# Cumulative PL by market to visually see trend and consistency
def bet_eval_chart_cPl(d):

    d = (
        d
        .groupby('market_id')
        .agg({'gpl': 'sum'})
    )

    d['market_number'] = np.arange(len(d))
    d['cGpl'] = d.gpl.cumsum()

    chart = px.line(d, x="market_number", y="cGpl", title='Cumulative Gross Profit', template='simple_white')

    return(chart)

def output_eval(summary):
    eval_df = bet_eval_metrics(summary)
    print(f'Total Gross Profit/Loss % = {(eval_df["gpl"]/eval_df["stake"]*100).values[0]}')
    print(eval_df)

def all_summary():
    dec = '2023_12'
    jan = '2024_01'
    feb = '2024_02'

    dir = 'trade_result_all_tracks' # change to trade_result_all_tracks

    df1 = pd.read_csv(f'{dir}/{dec}/{dec}_summary.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df1['month'] = dec

    df2 = pd.read_csv(f'{dir}/{jan}/{jan}_summary.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df2['month'] = jan

    df3 = pd.read_csv(f'{dir}/{feb}/{feb}_summary.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df3['month'] = feb

    summary = pd.concat([df1, df2, df3], axis=0)
    summary['gpl'] = summary['back_return'] + summary['lay_return']
    summary['stake'] = summary['back_v_sum'] + summary['lay_v_sum']
    
    # summary.to_csv(f'{dir}/all_months_summary.csv', index=False)

    eval_df = bet_eval_metrics(summary)
    print(f'Total Gross Profit/Loss % = {(eval_df["gpl"]/eval_df["stake"]*100).values[0]}')

    return summary

def all_raw_data():
    dec = '2023_12'
    jan = '2024_01'
    feb = '2024_02'

    dir = f'extracted_data' # change accordingly

    df1 = pd.read_csv(f'{dir}/{dec}/{dec}_preprocessed.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df1['month'] = dec

    df2 = pd.read_csv(f'{dir}/{jan}/{jan}_preprocessed.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df2['month'] = jan

    df3 = pd.read_csv(f'{dir}/{feb}/{feb}_preprocessed.csv',
                    dtype={'market_id': 'string', 'selection_id': 'string'},
                    )
    df3['month'] = feb

    raw = pd.concat([df1, df2, df3], axis=0)
    return raw
    # raw.to_csv(f'{dir}/all_months_raw_data.csv', index=False)

In [2]:
summary = all_summary()
raw = all_raw_data()

Total Gross Profit/Loss % = -2.3315104725506206


In [3]:
output_eval(summary)
bet_eval_chart_cPl(summary)

Total Gross Profit/Loss % = -2.3315104725506206
            gpl         stake       pot
0 -65479.438195  2.808456e+06 -0.023315


In [4]:
raw['vol'] = raw.groupby('market_id')['traded_volume'].diff().fillna(raw['traded_volume'])
venue_raw = raw.groupby('venue').agg({'vol': 'sum'}).reset_index()

venue_summary = summary.groupby('venue').agg({'gpl': 'sum'}).sort_values('gpl', ascending=True)

venue_summary = venue_summary.merge(venue_raw, on='venue', how='left')
venue_summary

,venue,gpl,vol
0,Port Lincoln,-11992.794479,106821.85
1,Murray Bridge,-8522.337212,93426.63
2,Ascot,-8157.338773,509395.84
3,Strathalbyn,-7254.183328,111443.89
4,Canberra,-6120.732458,138820.97
...,...,...,...
124,Seymour,6929.291009,95093.65
125,Kilcoy,7399.061659,12569.93
126,Geraldton,7432.397810,84515.09
127,Ballina,8069.026126,77907.54


In [5]:
venue_summary

,venue,gpl,vol
0,Port Lincoln,-11992.794479,106821.85
1,Murray Bridge,-8522.337212,93426.63
2,Ascot,-8157.338773,509395.84
3,Strathalbyn,-7254.183328,111443.89
4,Canberra,-6120.732458,138820.97
...,...,...,...
124,Seymour,6929.291009,95093.65
125,Kilcoy,7399.061659,12569.93
126,Geraldton,7432.397810,84515.09
127,Ballina,8069.026126,77907.54


In [7]:
from plotly.subplots import make_subplots

fig = make_subplots(specs=[[{"secondary_y": True}]])

# Add traces
fig.add_trace(
    go.Bar(x=venue_summary['venue'], y=venue_summary['gpl'], name="GPL"),
    secondary_y=False,
)
fig.add_trace(
    go.Scatter(x=venue_summary['venue'], y=venue_summary['vol'], name="Volume"),
    secondary_y=True,
)

# Add figure title
fig.update_layout(
    title_text="Total GPL and Volume by Venue"
)

# Set x-axis title
fig.update_xaxes(title_text="Venue")

# Set y-axes titles
fig.update_yaxes(title_text="<b>GPL</b>", secondary_y=False)
fig.update_yaxes(title_text="<b>Volume</b>", secondary_y=True)

tickvals = venue_summary['venue'].tolist()
ticktext = venue_summary['venue'].tolist()

fig.update_xaxes(tickvals=tickvals, ticktext=ticktext)
# Increase the graph size
fig.update_layout(width=1300, height=600)

# Reduce the x-axis label font size
fig.update_xaxes(tickfont=dict(size=8))

fig.show()